In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
import os
import json
from diffusers import StableDiffusionPipeline

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

print("Using GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


Using device: cuda
Using GPU: NVIDIA A100-SXM4-40GB


In [ ]:
from diffusers import StableDiffusion3Pipeline

pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3.5-large", torch_dtype=torch.bfloat16)
pipe = pipe.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:
import json
import os

# Directory where the image prompts are saved
outputs_dir = "/content/outputs"

# List all files in the outputs directory
files = os.listdir(outputs_dir)

# Filter for image prompt files
image_prompt_files = [f for f in files if 'image_prompt' in f and f.endswith('.json')]

print(f"Found {len(image_prompt_files)} image prompt files.")

# Create a directory to save images if it doesn't exist
os.makedirs("generated_images_stable_diffusion", exist_ok=True)

# Iterate over each image prompt file
for prompt_file in image_prompt_files:
    # Construct the full path to the prompt file
    prompt_file_path = os.path.join(outputs_dir, prompt_file)

    # Load the prompt data from the JSON file
    with open(prompt_file_path, 'r', encoding='utf-8') as f:
        prompt_data = json.load(f)

    # Extract the image prompt
    image_prompt = prompt_data.get('prompt')

    if not image_prompt:
        print(f"No prompt found in {prompt_file}")
        continue

    # Print the prompt for debugging
    print(f"Using prompt from {prompt_file}:\n{image_prompt}\n")

    # Generate the image using Stable Diffusion
    try:
        images = pipe(
            prompt=image_prompt,
            num_inference_steps=60,  # You can adjust this
            guidance_scale=8,      # You can adjust this
            num_images_per_prompt=1  # Generate multiple images
        ).images
    except Exception as e:
        print(f"Error generating image for {prompt_file}: {e}")
        continue

    # Get the product name from the filename
    product_name = '_'.join(prompt_file.split('_')[:-2])  # Adjust based on your filename pattern
    product_name = product_name.replace('_', ' ').title()  # Format the product name nicely

    # Save each generated image
    for idx, image in enumerate(images):
        image_filename = f"{product_name.replace(' ', '_').lower()}_stable_diffusion_image_{idx+1}.png"
        image_path = os.path.join("generated_images_stable_diffusion", image_filename)
        image.save(image_path)
        print(f"Image saved as '{image_path}'")

    print("-" * 50)


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['play.']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['play.']


Found 3 image prompt files.
Using prompt from handpan_drum_image_prompt_20241205063108.json:
The Handpan Drum is a 22-inch, 10-note musical instrument made from 1.2mm thick nitrogen steel. It's manually tuned for precise tones, producing an ethereal sound ideal for music education and meditation. The drum comes with a carrying case, stand, drumsticks, and cleaning cloth. It's durable, rust-resistant, and easy to play.



  0%|          | 0/60 [00:00<?, ?it/s]

Image saved as 'generated_images_stable_diffusion/handpan_drum_image_stable_diffusion_image_1.png'
--------------------------------------------------
Using prompt from breville_espresso_machine_image_prompt_20241205063108.json:
The Breville BES870XL Espresso Machine is a stainless steel, all in one espresso machine with an integrated grinder. It offers dose control grinding, optimal water pressure, and can go from beans to espresso in under a minute. Ideal for home baristas, it requires additional accessories for optimal use.



  0%|          | 0/60 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['- month trial of meta quest +.']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['- month trial of meta quest +.']


Image saved as 'generated_images_stable_diffusion/breville_espresso_machine_image_stable_diffusion_image_1.png'
--------------------------------------------------
Using prompt from meta_quest_3_image_prompt_20241205063108.json:
The Meta Quest 3 is a next-gen VR device with a 512GB capacity. It features a sharper resolution of 2064 x 2208 pixels per eye, a powerful Snapdragon XR2 Gen 2 chip, and full-color passthrough cameras for mixed reality experiences. It also includes a promotional offer for Batman: Arkham Shadow and a 3-month trial of Meta Quest+.



  0%|          | 0/60 [00:00<?, ?it/s]

Image saved as 'generated_images_stable_diffusion/meta_quest_3_image_stable_diffusion_image_1.png'
--------------------------------------------------
